<a href="https://colab.research.google.com/github/Siddharthgolecha/HighQ/blob/add_colab_notebook/QOSF_Mentorship_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit
!pip install torch
!pip install pennylane

     |████████████████████████████████| 6.1 MB 8.8 MB/s 
     |████████████████████████████████| 17.9 MB 112 kB/s 
     |████████████████████████████████| 235 kB 72.2 MB/s 
     |████████████████████████████████| 207 kB 74.4 MB/s 
     |████████████████████████████████| 2.1 MB 44.2 MB/s 
     |████████████████████████████████| 200 kB 61.7 MB/s 
     |████████████████████████████████| 634 kB 56.1 MB/s 
     |████████████████████████████████| 1.4 MB 58.0 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 38.2 MB 25 kB/s 
     |████████████████████████████████| 943 kB 45.4 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 3.0 MB 39.7 MB/s 
     |████████████████████████████████| 6.3 MB 39.6 MB/s 
  Created wheel for qiskit: filename=qiskit-0.30.0-py3-none-any.whl size=11273 sha256=6dd0c91060748d1d6345b0ec9283db665444f4b734561ab25906e9747065d418
  Stored in directory: /root/.cache/pip/whe

In [291]:
import qiskit
import numpy as np
import torch
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer
from qiskit.tools.visualization import plot_histogram
from torch.autograd import Function
from qiskit.visualization import *
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [281]:
class TrainingData:

  def __init__(self,n, size, backend=Aer.get_backend('qasm_simulator'),
               shots=1024):
    self.n = n
    self.backend = backend
    self.shots = shots
    self.size = size
    self.n_params = None
    self.samples = [self.generate_sample(self.n, self.backend, self.shots)
     for i in range(self.size)]

  def generate_sample(self,n,backend,shots):
    circuit = QuantumCircuit(n,n)
    for i in range(n):
      args = np.random.rand(3)*2*np.pi
      circuit.u(args[0], args[1], args[2], i)
      circuit.measure(i,i)
    result = backend.run(circuit, shots=shots).result()
    counts = result.get_counts()
    return max(counts, key = lambda x: counts[x])

  def to_params(self, n_params=None):
    if n_params is None:
      if n_params == self.n_params:
        raise "Add the number of paramters to convert"
      else:
        self.n_params = n_params
    self.params = np.zeros((self.size,n_params))
    for index in range(self.size):
      sample = self.samples[index]
      for i in range(self.n):
        state = sample[i]
        if state=="1":
          self.params[index][self.n-i-1] = np.pi/2
    return self.params

  def __str__(self):
    return str(self.samples)

In [282]:
class Labels:

  def __init__(self, data):
    self.data = data
    self.size = len(data)
    self.n = len(data[0])
    self.n_params = None

  def to_params(self, n_params=None):
    if n_params is None:
      if n_params == self.n_params:
        raise "Add the number of paramters to convert"
      else:
        self.n_params = n_params
    self.params = np.zeros((self.size,n_params))
    for index in range(self.size):
      sample = self.data[index]
      for i in range(self.n):
        state = sample[i]
        if state=="1":
          self.params[index][self.n-i-1] = np.pi/2
    return self.params

  def __str__(self):
    return str(self.data)

In [343]:
class QNNCircuit:
    """ 
    This class provides a simple interface for interaction 
    with the quantum circuit for training
    """
    
    def __init__(self, kernel_size,
                 backend=Aer.get_backend('qasm_simulator'),
                 shots = 1024):
        # --- Circuit definition ---
        self.kernel_size = kernel_size
        self.control_qubit = QuantumRegister(1, name="control")
        self.training_qubits = QuantumRegister(kernel_size, name="training")
        self.data_qubits = QuantumRegister(kernel_size, name="data")
        self.training_meas_register = ClassicalRegister(kernel_size,
                                                        "train_meas")
        self.measuring_register = ClassicalRegister(1, "meas")
        # ---------------------------
        self.backend = backend
        self.shots = shots
    
    def create_circuit(self, thetas, X):
      self.circuit = QuantumCircuit(self.control_qubit,
                                      self.training_qubits,
                                      self.data_qubits)
      for i in range(len(thetas)):
        if (i//self.kernel_size)%self.kernel_size == 0:
          self.circuit.ry(thetas[i], 
                          self.training_qubits[i%self.kernel_size])
        else:
          self.circuit.rz(thetas[i], 
                          self.training_qubits[i%self.kernel_size])
        
      for i in range(len(X)):
        if (i//self.kernel_size)%self.kernel_size == 0:
          self.circuit.ry(X[i], self.data_qubits[i%self.kernel_size])
        else:
          self.circuit.rz(X[i], self.data_qubits[i%self.kernel_size])
      self.circuit.h(self.control_qubit)
      for i in range(self.kernel_size):
        self.circuit.cswap(self.control_qubit, self.training_qubits[i],
                             self.data_qubits[i])
      self.circuit.h(self.control_qubit)
      self.train_circuit = self.circuit.copy()
        
    def expval(self):
      self.circuit.add_register(self.measuring_register)
      self.circuit.measure(self.control_qubit, self.measuring_register)
      job = self.backend.run(self.circuit, shots=self.shots)
      counts = job.result().get_counts()
      self.result = {int(key,2):val/self.shots for key,val in counts.items()}
      self.expval = 0
      for key,val in self.result.items():
        self.expval += key*val
      return self.expval

    def get_training_states(self):
      self.train_circuit.barrier()
      self.train_circuit.add_register(self.training_meas_register)
      self.train_circuit.measure(self.data_qubits,
                                 self.training_meas_register)
      job = self.backend.run(self.train_circuit, shots=self.shots)
      counts = job.result().get_counts()
      print(counts)
      output = {key: val/self.shots for key, val in counts.items()}
      return output

In [292]:
def get_accuracy(pred, actual):
  assert len(pred) == len(actual)
  total = 0
  for val in range(len(actual)):
    total += pred[i][actual] 
  return total/len(actual)

In [320]:
def paramList_to_numpy(params):
  new_params = np.array([param.data.numpy() for param in params])
  return new_params

In [366]:
class QNNFunction(Function):
    """ Hybrid quantum - classical function definition """
    
    @staticmethod
    def forward(ctx, data, params, quantum_circuits, kernel_size,
                shift, backend, shots):
        """ Forward pass computation """
        ctx.kernel_size = kernel_size
        ctx.shift = shift
        ctx.quantum_circuits = quantum_circuits
        ctx.backend = backend
        ctx.shots = shots
        output_classes = [ctx.quantum_circuits[i].get_training_states() 
        for i in range(data.n)]
        result = torch.tensor(output_classes)
        ctx.save_for_backward(data, params)

        return result
        
    @staticmethod
    def backward(ctx, grad_output):
        """ Backward pass computation """
        data, params = ctx.saved_tensors
        input_list = paramList_to_numpy(self.params)
        data_params = data.to_params()
        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift
        gradients = []
        right_circuit = [QNNCircuit(ctx.kernel_size) for i in
                         range(len(input_list))]
        left_circuit = [QNNCircuit(ctx.kernel_size) for i in
                         range(len(input_list))]
        for i in range(len(input_list)):
          right_circuit[i].create_circuit(shift_right[i], data_params[i])
          left_circuit[i].create_circuit(left_right[i], data_params[i])
          expectation_right = right_circuit[i].expval()
          expectation_left  = left_circuit[i].expval()
          gradient = torch.tensor(expectation_right) - torch.tensor(expectation_left)
          gradients.append(gradient.tolist())
        
        gradients = torch.tensor(gradients, dtype=torch.float64)
        gradients = torch.transpose(gradients,0,1)
        
        mm = torch.mm(grad_output,gradients)
        #print(mm.shape)
        return mm

In [367]:
class QNNModel(nn.Module):
    """ Hybrid quantum - classical layer definition """
    
    def __init__(self,kernel_size, data_size,
                 backend = Aer.get_backend('qasm_simulator'),
                 shots = 1024, shift = 0.01, n_params = 8):
        super(QNNModel, self).__init__()
        self.backend = backend
        self.kernel_size = kernel_size
        self.shots = shots
        self.shift = shift
        self.n_params = n_params
        self.data_size = data_size
        self.params = nn.ParameterList(
            [nn.Parameter(torch.rand(self.n_params)*2*np.pi)
             for i in range(self.kernel_size)])
        
    def forward(self, data):
        self.quantum_circuits = [QNNCircuit(self.kernel_size)
        for i in range(self.data_size)]
        params = paramList_to_numpy(self.params)
        data_params = data.to_params(self.n_params)
        for i in range(self.data_size):
          self.quantum_circuits[i].create_circuit(params[i],
                                                  data_params[i].tolist())
        return QNNFunction.apply(data, self.params, self.quantum_circuits,
                                    self.kernel_size, self.shift,
                                    self.backend, self.shots)

In [332]:
random_states = TrainingData(4, size=4)
print(random_states)
labels = Labels(['0011', '0101', '1010', '1100'])

['1100', '0001', '0011', '1010']


In [368]:
model= QNNModel(kernel_size=4, data_size=random_states.n, n_params=16)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().to(device)
epochs = 50
start_epoch = 0
loss_list = []
time_elapsed = []
accuracy_list = []

model.train()
for epoch in range(start_epoch,epochs):
    total_loss = []
    t = tqdm(labels.data,desc=f"Training: Epoch {epoch}")
    optimizer.zero_grad()
    # Forward pass
    output = model(random_states)
    print(output)
    # Calculating loss
    loss = criterion(output, labels.data)
    loss.backward()
    # Optimize the weights
    optimizer.step()
    loss_list.append(sum(loss.item())/len(loss.item()))
    time = t.format_interval(t.format_dict['elapsed'])
    time_elapsed.append(time)
    training_accuracy = get_accuracy(output, labels.data)
    accuracy_list.append(training_accuracy)
    print('Training [{:.0f}%]\tLoss: {:.4f}\tAccuracy: {:.4f}\tTime Taken: {}'.
          format(100. * (epoch + 1) / epochs, loss_list[-1],
                 training_accuracy, time))


Training: Epoch 0:   0%|          | 0/4 [01:21<?, ?it/s]

{'1101': 14, '1100': 132, '0101': 6, '1000': 137, '0110': 39, '0100': 128, '1111': 251, '0000': 110, '0111': 77, '1110': 130}
{'1101': 7, '1110': 2, '1011': 27, '1000': 245, '0001': 270, '1111': 3, '1010': 37, '1001': 149, '1100': 13, '0000': 271}
{'1110': 18, '1111': 99, '1101': 84, '0111': 130, '0011': 131, '0000': 138, '0010': 130, '1100': 19, '0101': 147, '0001': 94, '0110': 23, '0100': 11}
{'1101': 9, '0110': 1, '0100': 6, '0010': 135, '0000': 117, '1010': 130, '1000': 129, '1100': 432, '1110': 65}


RuntimeError: ignored